# Data -1 | Link

This file is the final step and performs a few closing operations. 

1. Add turnout to knn data.
2. Link voters across years.
3. Add Census variables (in progress). 

## Link KNN with Turnout

A list of elections in North Carolina https://en.wikipedia.org/wiki/Elections_in_North_Carolina.

In [9]:
elections

array(['11/03/2015 MUNICIPAL GENERAL', '11/04/2014 GENERAL',
       '05/06/2014 PRIMARY', '03/15/2016 PRIMARY', '11/06/2018 GENERAL',
       '10/10/2017 MUNICIPAL PRIMARY', '06/07/2016 CONGRESSIONAL PRIMARY',
       '11/07/2017 MUNICIPAL GENERAL', '05/08/2018 PRIMARY',
       '11/08/2016 GENERAL', '11/03/2020 GENERAL', '11/06/2012 GENERAL',
       '09/10/2019 GENERAL', '03/03/2020 PRIMARY', '05/08/2012 PRIMARY',
       '11/05/2013 MUNICIPAL GENERAL', '10/08/2013 MUNICIPAL PRIMARY',
       '11/05/2019 MUNICIPAL GENERAL', '05/14/2019 PRIMARY',
       '10/08/2019 MUNICIPAL PRIMARY', '07/17/2012 SECOND PRIMARY',
       '10/06/2015 MUNICIPAL PRIMARY', '11/05/2019 GENERAL',
       '04/30/2019 PRIMARY', '11/05/2019 MUNICIPAL',
       '11/02/2021 MUNICIPAL', '07/09/2019 SECOND PRIMARY',
       '11/02/2021 MUNICIPAL GENERAL', '11/05/2019 MUNICIPAL AND SCHOOL',
       '10/08/2019 HICKORY PRIMARY', '11/03/2015 MUNICIPALS',
       '11/07/2017 MUNICIPALS', '11/05/2013 MUNICIPAL',
       '11/07/2017

In [19]:
from datalink import *

#history = pd.read_pickle(path_0 + 'history/turnout_history.pkl')
elections = history.election_desc.unique()

main = ['GENERAL', 'PRIMARY', 'MUNICIPAL']
municipal = ['MUNI', 'MUNI AND SANI DISTRICT']
school = ['EDUCATION', 'SCHOOL']
alchohol = ['ALCHOHOL', 'BEVERAGE', 'ABC']
referendum = ['REFERENDUM']

use_election_types = main
election_types = {election_type: [e for e in elections if election_type in e] for election_type in use_election_types}

run_dates = [d for d in dates if int(d[:4])>=2010]
for date in run_dates:
    print(date, ' || ', ', '.join(election_types))
    year = date[:4]
    
    chunk_files = [x for x in os.listdir(path_1) if date in x]
    chunks = [x.strip(date) for x in chunk_files]
    
    """ Check If Everything Has Been Run """
    
    last_election_type = [e for e in election_types][-1]
    max_chunk = max([x for x in np.arange(0,30) if str(x) in '_'.join(chunks)])
    last_knn_chunk_save_name = f'knn_{date}_{last_election_type}_chunk_{max_chunk}.pkl'
    
    if last_knn_chunk_save_name not in os.listdir(path_link + 'chunks'):
        for chunk_name in sorted(chunk_files):
            
            #chunk = pd.read_pickle(path_1 + chunk_name)
            chunk_number = chunk_name.strip(f'{date}_').strip('.pkl')
            knn_chunk_name = 'knn_'+chunk_name.replace('chunk', f'{k_max}k_chunk')
            knn_chunk = pd.read_pickle(path_4 + 'chunks/' + knn_chunk_name)
            
            knn_chunk['white'] = (knn_chunk['race'] == 'WHITE')*1
            knn_chunk['black'] = (knn_chunk['race']=='BLACK or AFRICAN AMERICAN')*1
            #demographic_cols = ['white', 'black']
            
            print('   ', chunk_number, ' || ', len(knn_chunk), 'voters')#, len(chunk))
            
            for election_type in election_types:
                knn_chunk_save_name = 'knn_' + chunk_name.replace('chunk',f'{election_type}_chunk')
                
                if knn_chunk_save_name not in os.listdir(path_link + 'chunks'):
                    party_cols = [c for c in knn_chunk.columns.values if ('D' == c[0]) | ('R' == c[0]) | ('O' == c[0]) | ('white' in c) | ('black' in c)]

                    election_cols = [c for c in knn_chunk.columns.values if 'voted_' in c]
                    election_cols = [c for c in election_cols if election_type in c]
                    election_cols = [c for c in election_cols if (year in c) or (str(int(year) - 1) in c) or (str(int(year) + 1) in c)]
                    
                    keep_election_cols = []
                    for col in knn_chunk.columns.values:
                        if col in election_cols:
                            if sum(knn_chunk[col]) > 0:
                                keep_election_cols.append(col)
                                        
                    clean_knn_chunk = knn_chunk[['idu'] + party_cols + keep_election_cols]#+ demographic_cols
                    clean_knn_chunk.to_pickle(path_link + f'chunks/{knn_chunk_save_name}')
                    
    else:
        print('  Done')

20101102  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20110101  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20121106  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20130101  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20141231  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20151103  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20161108  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20171107  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20181106  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20191105  ||  GENERAL, PRIMARY, MUNICIPAL
  Done
20201103  ||  GENERAL, PRIMARY, MUNICIPAL
    chunk_0  ||  275267 voters
['voted_09/10/2019 GENERAL', 'voted_11/03/2020 GENERAL', 'voted_11/05/2019 MUNICIPAL GENERAL', 'voted_11/02/2021 MUNICIPAL GENERAL']
['voted_04/30/2019 PRIMARY', 'voted_07/09/2019 SECOND PRIMARY', 'voted_05/14/2019 PRIMARY', 'voted_03/03/2020 PRIMARY', 'voted_06/23/2020 SECOND PRIMARY', 'voted_10/05/2021 BURLINGTON PRIMARY', 'voted_10/08/2019 BURLINGTON PRIMARY']
['voted_11/05/2019 MUNICIPAL GENERAL', 'voted_11/05/2019 MUNICIPAL', 'v

## Link Year Sets


In [1]:
from datalink import *

use_this_k = '10050'
election_type = 'GENERAL'
base_cols = ['idu']#, 'first', 'middle', 'last', 'age', 'gender', 'race', 'address', 'birthyear']
k_cols = [p+str(k) for k in [0]+k_list for p in party_list]
base_cols = base_cols + party_list + k_cols

two_year_pairs = [['2010', '2012'], ['2012', '2014'], ['2014', '2016'], ['2016', '2018'], ['2018', '2020']]
four_year_pairs = [['2010', '2014'], ['2012', '2016'], ['2014', '2018'], ['2016', '2020']]
year_pairs = two_year_pairs + four_year_pairs

two_year_triples = [['2010', '2012', '2014'], ['2012', '2014', '2016'], ['2014', '2016', '2018'], ['2016', '2018', '2020']]
four_year_triples = [['2010', '2014', '2018'], ['2012', '2016', '2020']]
year_triples = two_year_triples + four_year_triples

years_list = year_pairs + year_triples

for years in years_list:
    year_name = '_'.join(years)
    dates = [date_map[year] for year in years]
    print(dates)
    
    if f'link_{year_name}' not in os.listdir(path_link):
    
        """ Import Years """

        voter_dict = {date:voters_from_chunks(path_link + 'chunks/', date + '_' + election_type) for date in dates}
        for year,date in zip(years,dates):
            voter_dict[date].index = voter_dict[date].idu
            election_cols = [col for col in voter_dict[date].columns.values if ('voted_' in col and year in col and 'GENERAL' in col)]
            keep_cols = base_cols + election_cols

            print('  ', year)
            print('  ', election_cols)

            voter_dict[date] = voter_dict[date][keep_cols]
            voter_dict[date] = voter_dict[date].rename(columns = {col:col+'_'+year for col in voter_dict[date]})

        """ Merge Years """

        link = pd.concat([voter_dict[date] for date in voter_dict], axis=1, join="inner")
        print('  ', len(link))
        
        link.to_pickle(path_link + f'link_{year_name}')

        del link
        del voter_dict
    else:
        print('  done')

['20101102', '20121106']
  done
['20121106', '20141231']
  done
['20141231', '20161108']
  done
['20161108', '20181106']
  done
['20181106', '20201103']
  done
['20101102', '20141231']
  done
['20121106', '20161108']
  done
['20141231', '20181106']
  done
['20161108', '20201103']
  done
['20101102', '20121106', '20141231']
  done
['20121106', '20141231', '20161108']
  done
['20141231', '20161108', '20181106']
  done
['20161108', '20181106', '20201103']
  done
['20101102', '20141231', '20181106']
  done
['20121106', '20161108', '20201103']
  done


## ACS and Squares (in progress)

Add ACS interpolated data to knn samples


In [ ]:
state = 'NC'
linkpath = '../voters/data5_link/'+state+'/'

with open('../voters/data4_knn/'+state+'/key_id_subsamples.pkl','rb') as f: key_id_subsamples = pickle.load(f)
keys = [key for key in key_id_subsamples]
datadict = {}
link_ids = []
for key in keys:
    if key+'_plus.csv' in os.listdir(linkpath):
        datadict[key] = pd.read_csv(linkpath+key+'_plus.csv', low_memory=False)
    else:
        datadict[key] = pd.read_csv(linkpath+key+'.csv', low_memory=False)
    link_ids = link_ids + list(datadict[key].DOUBLE_ID.unique())
link_ids = list(set(link_ids)) # List of subsample DOUBLE_IDS

meterlist = [500]#,2000]
for meters in meterlist:
    datalist = []
    for year in nc_yearlist:
        with open('../voters/data3_neighborhoods/'+state+'/SNd_'+str(year)+'_'+str(meters)+'m.pkl','rb') as f: SNd = pickle.load(f)
        finished = []
        for ni in SNd:
            n = SNd[ni]
            n['YEAR'] = year
            n['M_'+str(meters)+'m'] = sum((n.GENDER == 'M')*1)
            n['F_'+str(meters)+'m'] = sum((n.GENDER == 'F')*1)
            n['BLACK_'+str(meters)+'m'] = sum((n.RACE == 'BLACK or AFRICAN AMERICAN')*1)
            n['WHITE_'+str(meters)+'m'] = sum((n.RACE == 'WHITE')*1)
            n['OTHER_'+str(meters)+'m'] = sum((~n.RACE.isin(['BLACK or AFRICAN AMERICAN','WHITE']))*1)
            n['D_'+str(meters)+'m'] = n.D.sum()
            n['R_'+str(meters)+'m'] = n.R.sum()
            n['O_'+str(meters)+'m'] = n.O.sum()
            n['MEAN_AGE_'+str(meters)+'m'] = n.AGE.astype(float).mean()
            print(year,round(len(finished)/len(SNd)*100,2),'%')
            clear_output(wait=True)
            finished.append(ni)
        SNfinal = [SNd[i] for i in SNd]
        SNfinal = pd.concat(SNfinal,ignore_index=True, sort=False)
        SNfinal['DOUBLE_ID'] = SNfinal['ID_NUM'] + SNfinal['ID_NUM_a']
        SNfinal = SNfinal[SNfinal.DOUBLE_ID.isin(link_ids)]
        datalist.append(SNfinal)

    linkdata = pd.concat(datalist)
    linkdata = linkdata.sort_values(by=['DOUBLE_ID','YEAR'])
    
    keys = ['move','switch','stay','move_switch']
    for key in keys:
        key_linkdata = linkdata[linkdata.DOUBLE_ID.isin(datadict[key].DOUBLE_ID.unique())]
        key_linkdata['DOUBLE_ID_YEAR'] = key_linkdata['DOUBLE_ID'] + '_' + key_linkdata['YEAR'].astype(str)
        key_linkdata_droplist = ['ID_NUM','CITY','STATE','ZIP_CODE','ID_NUM_a','GENDER','AGE','RACE','sn_i','lat','lon','YEAR','D','R','O']
        key_linkdata_keeplist = [col for col in key_linkdata.columns.values if col not in key_linkdata_droplist]
        key_linkdata = key_linkdata[key_linkdata_keeplist]

        datadict[key]['DOUBLE_ID_YEAR'] = datadict[key]['DOUBLE_ID'] + '_' + datadict[key]['YEAR'].astype(str)

        merge = datadict[key].merge(key_linkdata, left_on='DOUBLE_ID_YEAR', right_on='DOUBLE_ID_YEAR', how='outer')
        rename_dict = {x:x.replace('_x','') for x in squares_keep_cols if '_x' in x}
        merge = merge.rename(columns=rename_dict)

        merge.to_csv('data5_link/NC/'+key+'_plus.csv')